In [1]:
! which python

/home/gsaltintas/.conda/envs/dl/bin/python


In [2]:
from utils import Argument
import swisstopo_helpers as sw
import geopandas as gpd

args = Argument()
args.max_rows = 500

In [11]:
import numpy as np
import io
# install osmnx through pip, conda version is too old
# osmnx version should be 1.2.2 and shapely 2.0.0
import osmnx as ox
import matplotlib.pyplot as plt
import cairosvg

import swisstopo_helpers as sw
from pathlib import Path

import re
import json
from pathlib import Path
import numpy as np
import cv2
import io
from geojson import Feature, GeoJSON, Polygon
from shapely.geometry import Polygon as PolygonShapely
import shapely

In [150]:
url = sw.get_url(args)
response = sw.send_request(url)

In [159]:
frame = gpd.geodataframe.GeoDataFrame(response['features'])
while get_next_response(response):
    print(cnt)
    url = response['links'][-1]['href']
    print('next page')
    response = sw.send_request(url)
    frame = frame.append(response['features']).reset_index(drop=True)
    if len(frame) >= args.max_rows:
        break
print(f'Found {len(frame)} entries.')

0
next page
1
next page
2
next page
3
next page
4
next page
5
next page
6
next page
7
next page
8
next page
9
next page
10
next page
11
next page
12
next page
13
next page
14
next page
15
next page
16
next page
17
next page
18
next page
19
next page
20
next page
21
next page
22
next page
23
next page
24
next page
25
next page
26
next page
27
next page
28
next page
29
next page
30
next page
31
next page
32
next page
33
next page
34
next page
35
next page
36
next page
37
next page
38
next page
39
next page
40
next page
41
next page
42
next page
43
next page
44
next page
45
next page
46
next page
47
next page
48
next page
49
next page
50
next page
51
next page
52
next page
53
next page
54
next page
55
next page
56
next page
57
next page
58
next page
59
next page
60
next page
61
next page
62
next page
63
next page
64
next page
65
next page
66
next page
67
next page
68
next page
69
next page
70
next page
71
next page
72
next page
73
next page
74
next page
75
next page
76
next page
77
next p

KeyboardInterrupt: 

In [162]:
url

'https://data.geo.admin.ch/api/stac/v0.9/collections/ch.swisstopo.swissimage-dop10/items?bbox=5%2C46%2C10%2C48&cursor=cj0xJnA9c3dpc3NpbWFnZS1kb3AxMF8yMDIyXzI2NzMtMTI4Mw%3D%3D&datetime=2022-01-01%2F2022-12-05'

In [158]:
def get_next_response(response, key='next'):

    for d in response['links']:
        if d['rel'] == key:
            return d['href']
    return None
response['links']
get_next_response(response)

'https://data.geo.admin.ch/api/stac/v0.9/collections/ch.swisstopo.swissimage-dop10/items?bbox=5%2C46%2C10%2C48&cursor=cD1zd2lzc2ltYWdlLWRvcDEwXzIwMjJfMjY3Ni0xMjUx&datetime=2022-01-01%2F2022-12-05'

In [104]:
from shapely.geometry import shape
from osmCategories import RailwayTags, BuildingTags, WaterTags, FieldTags
import time

counts = ['building', 'road', 'water', 'railway', 'field']

def get_stats(row):
#     if row.name > 0 and row.name % 50:
#         print('Waiting a little bit for API limits.')
#         time.sleep(5)
    bboxM = row['geometry']
    poly = shape(bboxM)

    buildings = ox.geometries_from_polygon(poly, tags=BuildingTags)
    roads_ = ox.graph_from_polygon(poly, retain_all=True, truncate_by_edge=False)
    
    try:
        waters = ox.geometries_from_polygon(
        poly, tags=WaterTags).groupby('waterway').count()
    except:
        waters = ox.geometries_from_polygon(
        poly, tags=WaterTags)
    roads = ox.utils_graph.graph_to_gdfs(roads_, nodes=False)
    railways = ox.geometries_from_polygon(poly, tags=RailwayTags)
    fields = ox.geometries_from_polygon(poly, tags=FieldTags)

    return {'building': len(buildings), 'road': len(roads), 'water': len(waters),
           'railway': len(railways),
           'field': len(fields)}
    
get_stats(frame.loc[0])

{'building': 3, 'road': 188, 'water': 3, 'railway': 0, 'field': 5}

In [163]:
coords = ['x_left', 'y_bottom', 'x_right', 'y_top']

for i, coord in enumerate(coords):
    frame[coord] = np.array(frame['bbox'].to_list())[:, i]
frame['x_center'] = (frame['x_left'] + frame['x_right'])/2
frame['y_center'] = (frame['y_bottom'] + frame['y_top']) /2
frame.head()


,id,collection,type,stac_version,geometry,bbox,properties,links,assets,"(x_left, y_bottom, x_right, y_top)",...,y_bottom,x_right,y_top,building,road,water,railway,field,x_center,y_center
0,swissimage-dop10_2022_2669-1250,ch.swisstopo.swissimage-dop10,Feature,0.9.0,"{'type': 'Polygon', 'coordinates': [[[8.366034...","[8.3526327, 47.3971037, 8.3660344, 47.4062027]","{'datetime': '2022-01-01T00:00:00Z', 'created'...","[{'rel': 'self', 'href': 'https://data.geo.adm...",{'swissimage-dop10_2022_2669-1250_0.1_2056.tif...,"[8.3526327, 47.3971037, 8.3660344, 47.4062027]",...,47.397104,8.366034,47.406203,3.0,188.0,3.0,0.0,5.0,8.359334,47.401653
1,swissimage-dop10_2022_2669-1257,ch.swisstopo.swissimage-dop10,Feature,0.9.0,"{'type': 'Polygon', 'coordinates': [[[8.367131...","[8.3537137, 47.460058, 8.3671316, 47.469157]","{'datetime': '2022-01-01T00:00:00Z', 'created'...","[{'rel': 'self', 'href': 'https://data.geo.adm...",{'swissimage-dop10_2022_2669-1257_0.1_2056.tif...,"[8.3537137, 47.460058, 8.3671316, 47.469157]",...,47.460058,8.367132,47.469157,7.0,140.0,1.0,0.0,13.0,8.360423,47.464607
2,swissimage-dop10_2022_2669-1258,ch.swisstopo.swissimage-dop10,Feature,0.9.0,"{'type': 'Polygon', 'coordinates': [[[8.367288...","[8.3538684, 47.4690514, 8.3672885, 47.4781504]","{'datetime': '2022-01-01T00:00:00Z', 'created'...","[{'rel': 'self', 'href': 'https://data.geo.adm...",{'swissimage-dop10_2022_2669-1258_0.1_2056.tif...,"[8.3538684, 47.4690514, 8.3672885, 47.4781504]",...,47.469051,8.367289,47.478150,8.0,80.0,1.0,0.0,5.0,8.360578,47.473601
3,swissimage-dop10_2022_2669-1259,ch.swisstopo.swissimage-dop10,Feature,0.9.0,"{'type': 'Polygon', 'coordinates': [[[8.367445...","[8.3540231, 47.4780447, 8.3674455, 47.4871437]","{'datetime': '2022-01-01T00:00:00Z', 'created'...","[{'rel': 'self', 'href': 'https://data.geo.adm...",{'swissimage-dop10_2022_2669-1259_0.1_2056.tif...,"[8.3540231, 47.4780447, 8.3674455, 47.4871437]",...,47.478045,8.367446,47.487144,5.0,70.0,1.0,0.0,8.0,8.360734,47.482594
4,swissimage-dop10_2022_2669-1260,ch.swisstopo.swissimage-dop10,Feature,0.9.0,"{'type': 'Polygon', 'coordinates': [[[8.367602...","[8.3541778, 47.487038, 8.3676026, 47.496137]","{'datetime': '2022-01-01T00:00:00Z', 'created'...","[{'rel': 'self', 'href': 'https://data.geo.adm...",{'swissimage-dop10_2022_2669-1260_0.1_2056.tif...,"[8.3541778, 47.487038, 8.3676026, 47.496137]",...,47.487038,8.367603,47.496137,5.0,70.0,1.0,0.0,25.0,8.360890,47.491587


In [133]:
for i in range(0, len(frame)):
    try:
        frame.loc[i, counts] = get_stats(frame.loc[i])
    except Exception as e:
        print(f'Error happened at index {i}: {e}')
    if i > 0 and i % 75 == 0:
        time.sleep(5)
        print('Waiting a little bit for API limits.')
      

Error happened at index 36: Server returned
<Response [408]> Request Timeout
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>408 Request Timeout</title>
</head><body>
<h1>Request Timeout</h1>
<p>Server timeout waiting for the HTTP request from the client.</p>
<hr>
<address>Apache/2.4.18 (Ubuntu) Server at overpass-api.de Port 443</address>
</body></html>

Waiting a little bit for API limits.
Error happened at index 82: Server returned
<Response [408]> Request Timeout
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>408 Request Timeout</title>
</head><body>
<h1>Request Timeout</h1>
<p>Server timeout waiting for the HTTP request from the client.</p>
<hr>
<address>Apache/2.4.18 (Ubuntu) Server at overpass-api.de Port 443</address>
</body></html>

Waiting a little bit for API limits.


In [138]:
import time
import datetime
frame.to_csv(f'swissimage-stats-{datetime.datetime.now()}.csv')

In [142]:
frame.describe()

,x_left,y_bottom,x_right,y_top,building,road,water,railway,field
count,200.000000,200.000000,200.000000,200.000000,197.000000,197.000000,197.000000,197.000000,197.000000
mean,8.409334,47.469893,8.422764,47.478998,146.573604,266.446701,1.954315,0.309645,11.030457
std,0.024171,0.116241,0.024177,0.116241,203.136957,320.465674,4.808893,1.638366,16.424816
min,8.352633,47.288400,8.366034,47.297509,0.000000,24.000000,0.000000,0.000000,0.000000
25%,8.393133,47.387416,8.406553,47.396525,12.000000,80.000000,1.000000,0.000000,3.000000
50%,8.410947,47.459564,8.424432,47.468669,50.000000,130.000000,1.000000,0.000000,6.000000
75%,8.431140,47.522712,8.444542,47.531815,197.000000,306.000000,2.000000,0.000000,13.000000
max,8.445518,47.738170,8.458938,47.747277,1115.000000,1591.000000,67.000000,20.000000,144.000000


In [53]:
row = frame.loc[0]

In [59]:
res

0     {'buildings': 3, 'roads': 188, 'water': 3, 'ra...
1     {'buildings': 7, 'roads': 140, 'water': 1, 'ra...
2     {'buildings': 8, 'roads': 80, 'water': 1, 'rai...
3     {'buildings': 5, 'roads': 70, 'water': 1, 'rai...
4     {'buildings': 5, 'roads': 70, 'water': 1, 'rai...
5     {'buildings': 30, 'roads': 76, 'water': 1, 'ra...
6     {'buildings': 90, 'roads': 303, 'water': 1, 'r...
7     {'buildings': 69, 'roads': 160, 'water': 2, 'r...
8     {'buildings': 105, 'roads': 804, 'water': 1, '...
9     {'buildings': 141, 'roads': 204, 'water': 1, '...
10    {'buildings': 197, 'roads': 289, 'water': 1, '...
dtype: object